In [8]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [9]:
%pip install torchinfo
%restart_python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\haroo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
UsageError: Line magic function `%restart_python` not found.


In [2]:
import os

from bacp import BaCPLearner, BaCPTrainer, BaCPTrainingArgumentsLLM
from models import EncoderProjectionNetwork, ClassificationNetwork
from unstructured_pruning import MagnitudePrune, MovementPrune, LocalMagnitudePrune, LocalMovementPrune, WandaPrune, PRUNER_DICT, check_model_sparsity
from LLM_trainer import LLMTrainer, LLMTrainingArguments
from dataset_utils import get_glue_data
from logger import Logger

import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset 

from tqdm import tqdm
from torchinfo import summary

from datasets.utils.logging import disable_progress_bar
disable_progress_bar()
os.environ["HF_DATASETS_CACHE"] = "/dbfs/hf_datasets"
os.environ["TOKENIZERS_PARALLELISM"] = "false" 

from utils import *
from constants import *

device = get_device()
print(f"{device = }")
BATCH_SIZE_DISTILBERT = 64
NUM_WORKERS = 24


device = 'cpu'


In [4]:
dataset = load_dataset("rajpurkar/squad")

In [5]:
print(dataset)
trainset = dataset['train']
valset = dataset['validation']

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})


In [18]:
trainset['answers'][0]

{'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}

In [12]:
from transformers import AutoTokenizer, DistilBertForQuestionAnswering
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased")

question, answer = "Who was Jim Henson?", "Jim Henson was a nice puppet"
example = {
    'question': question,
    'answer': answer,
}

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def tokenize_fn(example):
    return tokenizer(example['question'], example['answer'], padding='max_length', truncation=True, return_tensors='pt')

inputs = tokenize_fn(example)
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

# target is "nice puppet"
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss
round(loss.item(), 2)

6.23

### Create Dataloaders Here:

In [ ]:
# Use this as an example to make squad dataset.
# It wont follow the same logic, so dont copy paste it.

class GlueDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        return {
            "input_ids": example["input_ids"],
            "attention_mask": example["attention_mask"],
            "labels": example["label"]
        }

def get_glue_data(model_name, tokenizer, task_name, batch_size, num_workers=24):
    assert task_name in ["mnli", "qqp", "sst2"], f"Unsupported task: {task_name}"
    dataset = load_dataset("glue", task_name, cache_dir="/dbfs/hf_datasets")
    print(f"[DATALOADERS] {[key for key in dataset]}")

    def tokenize_fn(example):
        if task_name == "mnli":
            return tokenizer(example["premise"], example["hypothesis"], truncation=True, padding="max_length")
        if task_name == "qqp":
            return tokenizer(example["question1"], example["question2"], truncation=True, padding="max_length")
        if task_name == "sst2":
            return tokenizer(example["sentence"], truncation=True, padding="max_length")
        
    dataset = dataset.map(tokenize_fn, batched=True, batch_size=512, num_proc=1)
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

    trainset = GlueDataset(dataset["train"])
    valset = GlueDataset(dataset["validation"])
    testset = GlueDataset(dataset["test"])

    loader_args = {
        "batch_size" : batch_size,
        "num_workers" : num_workers,
        "pin_memory" : True,
        "persistent_workers" : num_workers > 0,
        "drop_last" : True,
    }

    trainloader = DataLoader(dataset["train"], shuffle=True, **loader_args)
    validationloader = DataLoader(dataset["validation"], **loader_args)
    testloader = DataLoader(dataset["test"], **loader_args)
  
    data = {
        "trainloader": trainloader,
        "trainset": trainset,
        "valloader": validationloader,
        "valset": valset,
        "testloader": testloader,
        "testset": testset
    }
    return data

In [ ]:
def get_squad_data(tokenizer, task_name, batch_size, num_workers=24):
    dataset = load_dataset("rajpurkar/squad")
    print(f"[DATALOADERS] {[key for key in dataset]}")

    # Tokenize inputs here - dekhlena how to do it
    # There are 2 sets - train and validation. Their size is too big, add some parameter to take a random subset.
    # tokenize it properly and return datasets and dataloaders.


### Create Training Script Here:

In [ ]:

# Follow the above example ive given to calculate the loss.
# Just make a generic training script.
# Main functionality should be that it calculates loss, back prop, and optimizer step so it learns, nothing fancy
# Ive given the bulk of it.

for data in dataloader:
    optimizer.zero_grad()

    inputs = ...
    with torch.no_grad():
        outputs = model(**inputs)

    answer_start_index = outputs.start_logits.argmax()
    answer_end_index = outputs.end_logits.argmax()

    predict_answer_tokens = inputs.input_ids[...]
    tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

    target_start_index = ...
    target_end_index = ...
    outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)

    loss = outputs.loss
    loss.backward()
    optimizer.step()